# How to load data
This notebook is to show how to load data from files

## Import packages

In [1]:
from pathlib import Path
from utils import check_integrity, datestring_to_timestamp, load_data, get_sorted_fluctuation_coins

## Set parameters and read data

In [2]:
data_path = "../data"
start_from = "2021-01-11"
end_before = "2021-03-15"
interval = "3h"

In [3]:
start_from_timestamp = datestring_to_timestamp(start_from)
end_before_timestamp = datestring_to_timestamp(end_before)
data_interval_path_list_raw = list(Path(data_path, interval).iterdir())
data_1D_path_list = list(Path(data_path, '1D').iterdir())

coins_1D_pass = []
for file_path in data_1D_path_list:
    _coin_code = file_path.name.split("_")[0]
    if check_integrity(start_from, end_before, file_path) is not None:
        coins_1D_pass.append(_coin_code)
print(f"{len(coins_1D_pass)} coins have all 1D data")

73 coins have all 1D data


In [4]:
data_interval_path_list = [_p for _p in data_interval_path_list_raw if _p.name.split('_')[0] in coins_1D_pass]

**Load the data of one coin from the given period**

In [5]:
_file_path = data_interval_path_list[0]

_data_df =  load_data(start_from_timestamp, end_before_timestamp, _file_path)
_data_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 505 entries, (1610323200000, 'BTC') to (1615766400000, 'BTC')
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   close     505 non-null    float64            
 1   volume    505 non-null    float64            
 2   datetime  505 non-null    datetime64[ns, UTC]
 3   is_fill   505 non-null    bool               
dtypes: bool(1), datetime64[ns, UTC](1), float64(2)
memory usage: 101.4+ KB


**Load another data of one coin from the given period then concatenate the data**

In [6]:
_file_path_2 = data_interval_path_list[9]

_data_df_2 =  load_data(start_from_timestamp, end_before_timestamp, _file_path_2)
_data_df_2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 503 entries, (1610323200000, 'MKR') to (1615766400000, 'MKR')
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   close     503 non-null    float64            
 1   volume    503 non-null    float64            
 2   datetime  503 non-null    datetime64[ns, UTC]
 3   is_fill   503 non-null    bool               
dtypes: bool(1), datetime64[ns, UTC](1), float64(2)
memory usage: 100.6+ KB


**Load and fill null values**

In [7]:
_data_df_2_filled =  load_data(start_from_timestamp, end_before_timestamp, _file_path_2, fill_na=True)
_data_df_2_filled.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 505 entries, (1610323200000, 'MKR') to (1615766400000, 'MKR')
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   close     505 non-null    float64            
 1   volume    505 non-null    float64            
 2   datetime  505 non-null    datetime64[ns, UTC]
 3   is_fill   505 non-null    bool               
dtypes: bool(1), datetime64[ns, UTC](1), float64(2)
memory usage: 50.3+ KB


**Concatenate dataframes**

In [8]:
_new_data_df = _data_df.append(_data_df_2)
_new_data_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1008 entries, (1610323200000, 'BTC') to (1615766400000, 'MKR')
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   close     1008 non-null   float64            
 1   volume    1008 non-null   float64            
 2   datetime  1008 non-null   datetime64[ns, UTC]
 3   is_fill   1008 non-null   bool               
dtypes: bool(1), datetime64[ns, UTC](1), float64(2)
memory usage: 47.9+ KB


**Sorted coins by fluctuation**

In [15]:
"""
get_sorted_fluctuation_coins(
    start_from (str, int): the timestamp or datestring of the first day
    end_before (str, int): the timestamp or datestring of the last day
    data_dir   (str):   the path if directory which has .csv files
    normalize_price (bool): use MinMaxScaler() to normalize the close price if True
    incl_coins (list, None): only return the coins in this provided list; None to return all coins
    return_details (bool): return the standard deviations in a DataFrame if True; False to only return a list with coin codes
    [DEPRECATED] start_from_timestamp (int): the timestamp of the first day
    [DEPRECATED] end_before_timestamp (int): the timestamp of the last day
)
"""

fluc_df = get_sorted_fluctuation_coins(
    start_from="2021-04-14",
    end_before="2021-06-15",
    data_dir='../data/1D',
    # incl_coins=ONLY_WANT_THESE_COINS,
    normalize_price=True,
    return_details=True,
    interval='1D'
)
fluc_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 119 entries, ORS to EOSDT
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   close_std   119 non-null    float64
 1   volume_sum  119 non-null    float64
dtypes: float64(2)
memory usage: 2.8+ KB


In [16]:
fluc_df.head()

,close_std,volume_sum
coin,,
ORS,0.353142,6.775591e+08
LUNA,0.352348,1.337474e+09
SAN,0.345045,1.630334e+09
QTF,0.344317,4.344900e+06
ICE,0.340424,4.391032e+07


In [17]:
fluc_df.tail()

,close_std,volume_sum
coin,,
BFT,0.174758,9.519669e+08
RRT,0.173251,2.366657e+08
EUS,0.168937,1.929136e+08
EUT,0.153312,9.153654e+07
EOSDT,0.127864,1.214146e+06


In [29]:
fluc_df[fluc_df['close_std']>0.3]

,close_std,volume_sum
coin,,
ORS,0.353142,6.775591e+08
LUNA,0.352348,1.337474e+09
SAN,0.345045,1.630334e+09
QTF,0.344317,4.344900e+06
ICE,0.340424,4.391032e+07
ETP,0.337074,1.099538e+09
BAL,0.335467,2.234116e+07
BEST,0.331503,1.421492e+07
FIL,0.325100,8.551556e+07


In [28]:
fluc_df[fluc_df['close_std']<0.25]

,close_std,volume_sum
coin,,
OCEAN,0.242045,6.968424e+07
SNX,0.241506,5.132308e+07
KAN,0.240600,1.827767e+09
UTK,0.240372,2.329116e+08
UOS,0.239854,2.444419e+09
MKR,0.239746,3.342947e+05
GNO,0.239318,6.626398e+04
DOGE,0.238783,1.410452e+11
ETC,0.238173,8.491384e+08
